In [ ]:
import pandas as pd

gps_data = pd.read_csv('clustered_trips.csv')
gps_data

,trip_id,distance,duration,avg_speed,start_time,end_time,start_location,end_location,halt_count,avg_halt_duration,...,max_halt_location,start_label,end_label,max_halt_label,start_time_category,end_time_category,start_time_day,end_time_day,gmm_cluster,kmeans_cluster
0,C001_06-Veloci2_1.0,5080.199201,19.950000,4.244110,2024-01-31 17:49:44,2024-01-31 18:09:41,"(52.66710166666667, -8.576183333333333)","(52.67005833333333, -8.633385)",6,33.791667,...,[52.66534 -8.607245],Commercial,Residential,Commercial,Afternoon,Evening,Wed,Wed,1,5
1,C001_06-Veloci2_110.0,6298.928335,19.500000,5.383699,2024-03-04 08:42:54,2024-03-04 09:02:24,"(52.67011, -8.633483333333333)","(52.667611666666666, -8.574975)",3,7.636364,...,[52.66794166666666 -8.576695],Residential,Commercial,Residential,Morning,Morning,Mon,Mon,3,2
2,C001_06-Veloci2_111.0,1765.016288,9.516667,3.091097,2024-03-04 10:18:56,2024-03-04 10:28:27,"(52.666605, -8.57684)","(52.672138333333336, -8.556015)",3,86.214286,...,[52.672138333333336 -8.556015],Recreational,Commercial,Commercial,Morning,Morning,Mon,Mon,3,2
3,C001_06-Veloci2_112.0,3399.361455,7.916667,7.156550,2024-03-04 12:21:25,2024-03-04 12:29:20,"(52.67209666666667, -8.555928333333334)","(52.671821666666666, -8.55602)",3,188.461538,...,[52.671821666666666 -8.55602],Commercial,Commercial,Commercial,Afternoon,Afternoon,Mon,Mon,0,4
4,C001_06-Veloci2_113.0,8880.359707,33.950000,4.359529,2024-03-04 18:16:24,2024-03-04 18:50:21,"(52.671821666666666, -8.55602)","(52.66999833333333, -8.63372)",8,41.325000,...,[52.670008333333335 -8.633391666666666],Commercial,Residential,Residential,Evening,Evening,Mon,Mon,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,P666_ZZ-JG_86.0,4132.682274,21.433333,3.213594,2023-09-20 15:43:41,2023-09-20 16:05:07,"(52.67801833333333, -8.569161666666666)","(52.666845, -8.543201666666667)",4,160.655172,...,[52.666845 -8.543201666666667],School,Residential,Residential,Afternoon,Afternoon,Wed,Wed,1,5
4654,P666_ZZ-JG_87.0,3149.144739,21.216667,2.473798,2023-09-20 16:22:05,2023-09-20 16:43:18,"(52.666205, -8.54393)","(52.66678666666667, -8.543111666666666)",5,119.760000,...,[52.65831333333333 -8.54979],Residential,Residential,School,Afternoon,Afternoon,Wed,Wed,1,5
4655,P666_ZZ-JG_88.0,3692.867836,24.033333,2.560935,2023-09-20 17:28:30,2023-09-20 17:52:32,"(52.66665833333333, -8.543538333333334)","(52.66679666666667, -8.543175)",5,211.750000,...,[52.66080333333333 -8.54976],Residential,Residential,Residential,Afternoon,Afternoon,Wed,Wed,0,4
4656,P666_ZZ-JG_89.0,3149.814093,16.933333,3.100211,2023-09-20 18:53:16,2023-09-20 19:10:12,"(52.66588166666666, -8.545276666666666)","(52.666855, -8.543191666666667)",6,72.350000,...,[52.65833666666666 -8.549711666666667],Recreational,Residential,Recreational,Evening,Evening,Wed,Wed,1,5


In [3]:
import pandas as pd
import networkx as nx
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
import numpy as np


In [4]:
import ast
import numpy as np

def parse_point(p):
    if isinstance(p, (tuple, list)):
        return [float(p[0]), float(p[1])]
    return list(ast.literal_eval(p))

gps_data['start_xy'] = gps_data['start_location'].apply(parse_point)
gps_data['end_xy']   = gps_data['end_location'].apply(parse_point)


# Collect all start & end points
coords = np.vstack([
    gps_data['start_xy'].to_list(),
    gps_data['end_xy'].to_list()
]).astype(float)

# DBSCAN spatial clustering (100m radius)
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian  # 100m

db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine')
labels = db.fit_predict(np.radians(coords))

# Assign cluster IDs back
n = len(gps_data)
gps_data['start_node'] = labels[:n]
gps_data['end_node'] = labels[n:]


In [35]:
G = nx.DiGraph()

selected_trips = gps_data[gps_data['gmm_cluster']==3]

for _, row in selected_trips.iterrows():
    u = row['start_node']
    v = row['end_node']
    if u == -1 or v == -1:
        continue
    
    if G.has_edge(u, v):
        G[u][v]['weight'] += 1
    else:
        G.add_edge(u, v, weight=1)


degree_dict = dict(G.degree())
hub_nodes = [n for n, d in degree_dict.items() if d > 20]

print("High-connectivity hubs:", hub_nodes)


High-connectivity hubs: [2, 3, 4, 12, 14, 19, 21, 30]


In [36]:
node_coords = {}

for node in set(labels):
    if node == -1:
        continue
    pts = coords[labels == node]
    node_coords[node] = pts.mean(axis=0)


In [ ]:
import folium

center = coords.mean(axis=0)
m = folium.Map(location=center, zoom_start=13)

for u, v, data in G.edges(data=True):
    if u in node_coords and v in node_coords:
        folium.PolyLine(
            locations=[node_coords[u], node_coords[v]],
            weight=1 + data['weight'],
            opacity=0.5
        ).add_to(m)

for node, (lat, lon) in node_coords.items():
    deg = degree_dict.get(node, 0)
    
    color = "red" if node in hub_nodes else "blue"
    radius = 8 if node in hub_nodes else 4

    folium.CircleMarker(
        location=[lat, lon],
        radius=radius,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=f"Node {node} | Degree: {deg}"
    ).add_to(m)

m.save("GMMCluster3.html")
print("Map saved as GMMCluster3.html")

Map saved as GMMCluster3.html


: 

In [14]:
gps_data[gps_data['gmm_cluster']==0][['start_label', 'end_label']].value_counts()

start_label     end_label         
Residential     Residential           79
School          Residential           44
                School                42
Commercial      Residential           37
Residential     School                30
                Commercial            24
Commercial      Commercial            19
School          Commercial            13
Residential     Transportation        13
Transportation  Residential           11
Residential     Recreational          10
Commercial      School                 8
Transportation  Transportation         7
Recreational    Residential            6
Commercial      Transportation         5
School          Transportation         4
Transportation  School                 4
Recreational    School                 2
School          Public Institution     2
                Recreational           2
Transportation  Commercial             2
Recreational    Transportation         1
Residential     Public Institution     1
Commercial      Recrea

In [ ]:
gps_data[gps_data['gmm_cluster']==0][['max_halt_label']].value_counts()

max_halt_label
School            129
Residential       120
Commercial         82
Transportation     23
Recreational       16
Name: count, dtype: int64

In [16]:
gps_data[gps_data['gmm_cluster']==1][['max_halt_label']].value_counts()

max_halt_label
Residential       568
School            372
Commercial        321
Transportation    182
Recreational       75
Industrial         19
Name: count, dtype: int64

In [ ]:
gps_data[gps_data['gmm_cluster']==1][['start_label', 'end_label']].value_counts()

start_label         end_label     
Residential         Residential       366
School              Residential       333
Commercial          Residential       206
Residential         Commercial         92
Transportation      Residential        81
School              School             75
Residential         School             69
Recreational        Residential        60
Residential         Recreational       39
School              Commercial         38
Commercial          Commercial         29
                    School             29
Residential         Transportation     22
Public Institution  Residential        21
Transportation      Transportation     15
Commercial          Transportation     12
School              Transportation     11
Recreational        Commercial          8
School              Recreational        7
Transportation      Commercial          5
Recreational        Recreational        4
                    Transportation      3
Industrial          Residential         3

In [21]:
gps_data[gps_data['gmm_cluster']==2][['max_halt_label']].value_counts()

max_halt_label    
Commercial            45
Residential           45
Public Institution    40
School                36
Religious             28
Transportation        20
Recreational          15
Name: count, dtype: int64

In [ ]:
gps_data[gps_data['gmm_cluster']==2][['start_label']].value_counts()

start_label       
Residential           104
Public Institution     34
School                 34
Commercial             32
Industrial             16
Recreational            5
Transportation          4
Name: count, dtype: int64

In [29]:
gps_data[gps_data['gmm_cluster']==0][['end_label']].value_counts()

end_label         
Residential           178
School                 86
Commercial             58
Transportation         30
Recreational           14
Public Institution      4
Name: count, dtype: int64

In [30]:
gps_data['end_label'].value_counts()

end_label
Residential           2367
School                1069
Commercial             729
Transportation         268
Recreational           127
Public Institution      73
Industrial              22
Religious                3
Name: count, dtype: int64

In [31]:
gps_data['start_label'].value_counts()

start_label
Residential           2399
School                1070
Commercial             699
Transportation         252
Recreational           154
Public Institution      57
Industrial              25
Religious                2
Name: count, dtype: int64